## ME18B156 - Assignment 3 - Transliteration

In [ ]:
!git clone https://github.com/Mukesh-V/Acads.git
%cd /content/Acads/DL/AS-3

In [ ]:
!pip install wandb pytorch-lightning Levenshtein

In [ ]:
!wandb online
%env WANDB_API_KEY=77d02595152611dfbdff0cd06134f43db0cf56f3

main.py has two arguments : learning rate and teacher-forcing. Both should lie between 0 and 1. Keeping the teacher-forcing higher reduces its effect (it is applied lesser times)

In [ ]:
!python3 main.py --lr=0.01 --tf=0.5

In [4]:
# loaded from wandb's best run
no_att_best_run = 'model-l1i1t17s:v10'
no_att_config = {
    'model': 'no-attention',
    'drop': 0.1,
    'embedding': 16,
    'unit': 'gru',
    'hidden': 256,
    'layers': 3,
    'epochs': 20
}

att_best_run = 'model-9y3fvg3a:v18'
att_config = {
    'model': 'attention',
    'drop': 0.05,
    'embedding': 16,
    'unit': 'gru',
    'hidden': 64,
    'layers': 3,
    'epochs': 30
}

In [ ]:
import wandb

%env WANDB_API_KEY=77d02595152611dfbdff0cd06134f43db0cf56f3

run = wandb.init()
no_attention_artifact = run.use_artifact('muttaborota/FundDL-AS3/' + no_att_best_run, type='model')
no_attention_artifact_dir = no_attention_artifact.download()

attention_artifact = run.use_artifact('muttaborota/FundDL-AS3/' + att_best_run, type='model')
attention_artifact_dir = attention_artifact.download()
wandb.run.finish()

In [ ]:
from data import TransliterationDataset
from torch.utils.data import DataLoader

batch_size = 512
testloader = DataLoader(TransliterationDataset('test'), batch_size, num_workers=2)

In [9]:
import torch
from model_no_att import Transliterator
from model_att import AttentionTransliterator

maps = testloader.dataset.maps

# Loading the models
class Struct:
    def __init__(self, **entries):
        self.__dict__.update(entries)
no_attention_struct = Struct(**no_att_config)
attention_struct = Struct(**att_config)

no_attention_model = Transliterator.load_from_checkpoint('./artifacts/' + no_att_best_run + '/model.ckpt', config=no_attention_struct, maps=maps, map_location=torch.device('cpu'), strict=False)
attention_model = AttentionTransliterator.load_from_checkpoint('./artifacts/' + att_best_run + '/model.ckpt', config=attention_struct, maps=maps, map_location=torch.device('cpu'), strict=False)

### Prediction and Accuracy

In [20]:
import torch
from torch.nn import Softmax

maps = testloader.dataset.maps
crct = 0
for i, batch in enumerate(testloader):
  x, y = batch
  preds = torch.argmax(Softmax(dim=2)(no_attention_model(x)), dim=2)       
  df = open('predictions_vanilla.txt', 'w') 

  for input, pred, truth in zip(x, preds, y):
    input_word = ''.join([maps['i2ic'][i.item()] for i in input]).replace('\t', '').replace('\n', '').replace(' ', '')
    pred_word = ''.join([maps['i2oc'][i.item()] for i in pred]).replace('\t', '').replace('\n', '').replace(' ', '')
    truth_word = ''.join([maps['i2oc'][i.item()] for i in truth]).replace('\t', '').replace('\n', '').replace(' ', '')

    if(pred_word == truth_word): crct += 1
    df.write('{} {} {} \n'.format(input_word, truth_word, pred_word))
  df.close()

print('Accuracy : ', crct/(len(x)*len(testloader)))

Accuracy :  0.15826810176125244


In [21]:
import torch
from torch.nn import Softmax

maps = testloader.dataset.maps
crct = 0
for i, batch in enumerate(testloader):
  x, y = batch
  preds = torch.argmax(Softmax(dim=2)(attention_model(x)), dim=2)  
  df = open('predictions_attention.txt', 'w') 

  for input, pred, truth in zip(x, preds, y):
    input_word = ''.join([maps['i2ic'][i.item()] for i in input]).replace('\t', '').replace('\n', '').replace(' ', '')
    pred_word = ''.join([maps['i2oc'][i.item()] for i in pred]).replace('\t', '').replace('\n', '').replace(' ', '')
    truth_word = ''.join([maps['i2oc'][i.item()] for i in truth]).replace('\t', '').replace('\n', '').replace(' ', '')

    if(pred_word == truth_word): crct += 1
    df.write('{} {} {} \n'.format(input_word, truth_word, pred_word))
  df.close()

print('Accuracy : ', crct/(len(x)*len(testloader)))

Accuracy :  0.14554794520547945
